In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path

def backtest(df: pd.DataFrame, price: int = 0.002):
    result_dict = {}
    pos = 0
    max_high = 0
    is_long_day_interval = False
    last_date = None
    for idx, bar_series in df.iterrows():
        if not pos:
            if bar_series['low'] > price:
                continue
            else:
                if bar_series['high'] > price:
                    result_dict['start'] = bar_series['date']
                    result_dict['entry'] = price * 10000
                    pos = 1
                    last_date = bar_series['date']
                    max_high = max(max_high, bar_series['close'])
        else:
            max_high = max(max_high, bar_series['high'])
            if not is_long_day_interval:
                if last_date:
#                     print('last_date', last_date)
#                     print('today', bar_series['date'])
                    if bar_series['date'] - last_date > timedelta(days=1):
                        is_long_day_interval = True
                last_date = bar_series['date']

    if pos:
        result_dict['code'] = df.iloc[-1]['code']
        result_dict['trading_code'] = df.iloc[-1]['trading_code']
        result_dict['end'] = df.iloc[-1]['date']
        result_dict['exit'] = df.iloc[-1]['close'] * 10000
        result_dict['max_high'] = max_high * 10000
        result_dict['duration'] = (result_dict['end'] - result_dict['start']).days
        result_dict['max_db'] = min(0, result_dict['entry'] - result_dict['max_high'])
        result_dict['long_date_gap'] = is_long_day_interval
    else:
        #         print('no position')
        return
        
#     print(result_dict)
    return result_dict

In [ ]:
# 载入数据
keep_items = ['code', 'date', 'open', 'high', 'low', 'close', 'volume', 'position', 'trading_code', 'status']

data_path = Path.cwd().joinpath('daily_bar')
files = list(data_path.glob('*.csv'))
files

df_list = [pd.read_csv(file, index_col=0, parse_dates=[4]) for file in files]
df = pd.concat(df_list, ignore_index=True)
df = df[keep_items]

# 去掉调整过以及交易中的合约
df2 = df[(df['trading_code'].map(lambda x: x[11] == 'M')) & (df['status'] == 'DELIST')].copy()

In [ ]:
# 关于最大回撤，因为当天触发交易和最高价的先后顺序从k线之中无法判断，因此当天下单后的最高价取当天收盘价，以后的交易日取最高价
res_list = []
for _code, bar_df in df2.groupby('code'):
    res_dict = backtest(bar_df)
    if res_dict:
        res_list.append(res_dict)
res_df = pd.DataFrame(res_list)

In [ ]:
# 保存结果
new_order = ['code', 'trading_code', 'start', 'end', 'duration', 'entry', 'exit', 'max_high', 'max_db', 'long_date_gap']
res_df = res_df[new_order]
res_df.to_csv(Path.cwd().joinpath('result', 'low_preminu.csv'))

In [ ]:
# 分析
days_within_2 = res_df[res_df['duration'] <= 3]
print("all count:", len(days_within_2))

days_within_2_no_db = days_within_2[days_within_2['max_db'] == 0]
print('without drawback:', len(days_within_2_no_db))

days_within_2.describe()

In [ ]:
# 测试代码
code = '10000081.XSHG'
test_df = df2[df2['code'] == code]
test_df

In [ ]:
res = backtest(test_df)
res